In [ ]:
import numpy as np

In [ ]:
def compareProbs(initial, actions):
  prob_matrix = []
  new_arr = initial
  for action in actions:
    new_arr = np.copy(new_arr)
    new_arr[action[0]] *= action[1]
    prob_matrix.append(new_arr)

  return prob_matrix

In [ ]:
actions = [[0, 1.5], [2, 1.5]]
initial = [10, 10, 10, 10]

In [ ]:
prob_matrix = compareProbs(initial, actions)
print(prob_matrix)

[array([15, 10, 10, 10]), array([15, 10, 15, 10])]


In [ ]:
for prob in prob_matrix:
  print(np.divide(prob, np.sum(prob)))

[0.33333333 0.22222222 0.22222222 0.22222222]
[0.3 0.2 0.3 0.2]


In this example, we see that modifiers such as 50% increased chance to show up likely does not mean that there is a 50% of their base chance. This likely corrobates with the data how you can have multiple objects with increased chance but given that each event is mutually exclusive, 50% increased chance must mean decreases for other events.

In [ ]:
def calculateTrueIncrease(initial, actions):
  prob_matrix = []
  new_arr = initial
  for action in actions:
    new_arr = np.copy(new_arr)
    new_arr = new_arr.astype(float)
    cur_sum = np.sum(new_arr)
    action_prob = (new_arr[action[0]]/cur_sum) * action[1]
    # print((new_arr[action[0]]/cur_sum), action_prob)
    added_weight = (new_arr[action[0]]- action_prob*cur_sum)/(action_prob-1)
    new_arr[action[0]] += added_weight
    # print(new_arr[action[0]]/np.sum(new_arr))
    prob_matrix.append(new_arr)
  return prob_matrix

In [ ]:
test = calculateTrueIncrease(initial, actions)
test

[array([18., 10., 10., 10.]),
 array([18.        , 10.        , 17.27272727, 10.        ])]

In [ ]:
for arr in test:
  print(arr/np.sum(arr))

[0.375      0.20833333 0.20833333 0.20833333]
[0.32565789 0.18092105 0.3125     0.18092105]


Within this example, I show how calculations for increased probability chance between various steps could also lead problems. Within this case element 1 and element 3 both have increased probability of 50%, but since we changed element 1 first to have an increased probability and then element 3, this led to different balancing then expected.

In [ ]:
def trueProb(initial, actions):
  lin_matrix = []
  action_added_dict = {}
  right_side = []
  initial = np.array(initial)
  initial = initial.astype(float)
  for idx, action in enumerate(actions):
    #left hand side
    cur_action_weight = initial[action[0]]
    desired_prob = (initial[action[0]]/np.sum(initial)) * action[1]
    cur_lin = np.ones(len(actions))
    cur_lin = np.multiply(cur_lin, -1 * desired_prob)
    cur_lin[idx] = 1 + cur_lin[idx]
    lin_matrix.append(cur_lin)

    #right hand side
    right_side.append(np.sum(initial) * desired_prob - initial[action[0]])
  added_values = np.linalg.solve(lin_matrix, right_side)
  for idx, action in enumerate(actions):
    initial[action[0]] += added_values[idx]
  
  return initial

In [ ]:
true_probs = trueProb(initial, actions)

In [ ]:
np.divide(true_probs, np.sum(true_probs))

array([0.375, 0.125, 0.375, 0.125])

The implementation on the top is how you would actually get the correct scaling. In order to do this, you have to set up linear equations to solve for each of the added values. Solving this linear system will give you what values to add for each changed variable, and results in the correct answer.